##

sub-21
File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/encoding_model/fit_nprf_cv_stim2.py", line 136, in <module>
main(args.subject, bids_folder=args.bids_folder, smoothed=args.smoothed, retroicor=args.retroicor,
File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/encoding_model/fit_nprf_cv_stim2.py", line 70, in main
`data = pd.DataFrame(masker.fit_transform(data), index=paradigm.index)`
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/frame.py", line 721, in __init__
mgr = ndarray_to_mgr(
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 349, in ndarray_to_mgr
_check_values_indices_shape_match(values, index, columns)
File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 420, in _check_values_indices_shape_match
raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (105, 74624), indices imply (180, 74624)

sub-3
....    _check_values_indices_shape_match(values, index, columns)
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/pandas/core/internals/construction.py", line 420, in _check_values_indices_shape_match
    raise ValueError(f"Shape of passed values is {passed}, indices imply {implied}")
ValueError: Shape of passed values is (104, 84111), indices imply (180, 84111)


###
for sub 3 this error comes already for fit_glm_stim1 ! (not stim2 though...)
different, thoug:
  File "/home/mrenke/git/numrisk/numrisk/fmri_analysis/glm_denoise/fit_glm_denoise.py", line 96, in main
    results_glmsingle = glmsingle_obj.fit(
  File "/home/mrenke/git/GLMsingle/glmsingle/glmsingle.py", line 608, in fit
    results0 = glm_estimatemodel(
  File "/home/mrenke/git/GLMsingle/glmsingle/ols/glm_estimatemodel.py", line 425, in glm_estimatemodel
    data, design, xyz = check_inputs(data, design)
  File "/home/mrenke/git/GLMsingle/glmsingle/check_inputs.py", line 27, in check_inputs
    np.testing.assert_array_equal(
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 934, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/data/mrenke/conda/envs/numrefields/lib/python3.9/site-packages/numpy/testing/_private/utils.py", line 763, in assert_array_compare
    raise AssertionError(msg)
AssertionError:
Arrays are not equal
<design> must consist of 0s and 1s
(shapes (3,), (2,) mismatch)
 x: array([0., 1., 2.])
 y: array([0, 1])


In [1]:
import argparse
import pandas as pd
from braincoder.models import GaussianPRF
from braincoder.optimize import ParameterFitter
from braincoder.utils import get_rsq
from numrisk.utils import get_target_dir
from nilearn.input_data import NiftiMasker

import os.path as op
import numpy as np

/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [26]:
bids_folder = '/shares/zne.uzh/mrenke/ds-dnumrisk'

subject = 3


subject = f'{int(subject):02d}'
session = 1
runs = range(1, 7)
n_stim = 2
key = f'glm_stim{n_stim}'
target_dir = f'encoding_model_stim{n_stim}.cv'
key += '.denoise'
target_dir += '.denoise'


In [27]:
target_dir = get_target_dir(subject, session, bids_folder, target_dir)

paradigm = [pd.read_csv(op.join(bids_folder, f'sub-{subject}', f'ses-{session}',
                                'func', f'sub-{subject}_ses-{session}_task-magjudge_run-{run}_events.tsv'), sep='\t')
            for run in runs]
paradigm = pd.concat(paradigm, keys=runs, names=['run'])
paradigm = paradigm[paradigm.trial_type == f'stimulus {n_stim}'].set_index('trial_nr', append=True)

paradigm[f'log(n{n_stim})'] = np.log(paradigm[f'n{n_stim}'])
paradigm = paradigm[f'log(n{n_stim})'].dropna()

In [28]:
paradigm

run      trial_nr
1    1   1           1.386294
     4   2           1.609438
     7   3           1.386294
     10  4           2.397895
     13  5           2.397895
                       ...   
6    76  176         3.496508
     79  177         2.197225
     82  178         2.639057
     85  179         1.098612
     88  180         2.397895
Name: log(n2), Length: 180, dtype: float64

In [29]:
model = GaussianPRF()
# SET UP GRID
mus = np.log(np.linspace(5, 80, 60, dtype=np.float32))
sds = np.log(np.linspace(2, 30, 60, dtype=np.float32))
amplitudes = np.array([1.], dtype=np.float32)
baselines = np.array([0], dtype=np.float32)

mask = op.join(bids_folder, 'derivatives',
                f'fmriprep/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-1_space-T1w_desc-brain_mask.nii.gz')
if not op.exists(mask): # for some subs brain mask creation for some runs did not work apparently, so lets just take another run (they should be pretty similar?)
    mask = op.join(bids_folder, 'derivatives',
            f'fmriprep/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-2_space-T1w_desc-brain_mask.nii.gz')

masker = NiftiMasker(mask_img=mask)



In [30]:
data = op.join(bids_folder, 'derivatives', key,
                f'sub-{subject}', f'ses-{session}', 'func', f'sub-{subject}_ses-{session}_task-magjudge_space-T1w_desc-stims{n_stim}_pe.nii.gz')

from nilearn import image

ims = image.load_img(data)

In [31]:
ims.get_fdata().shape

(58, 73, 50, 104)

In [32]:
tmp = masker.fit_transform(data)

In [21]:
tmp

array([[ -5.586601  ,  -0.25460613,  13.430651  , ...,   3.276642  ,
         -0.39079255,  -9.162464  ],
       [  0.17910606,   6.005245  ,  19.335033  , ...,   3.3928962 ,
         -1.3308599 ,   4.1230335 ],
       [  5.7466693 ,   1.1067244 , -10.965698  , ...,  -1.0369884 ,
         -2.8626223 ,   7.8606257 ],
       ...,
       [ -1.7437749 ,   3.8153033 ,   2.5822282 , ...,  -6.538029  ,
         -0.23672003, -41.86214   ],
       [ -7.4950514 ,  -5.620179  ,   1.7101481 , ...,  28.328648  ,
         -6.5455327 ,  88.55697   ],
       [ -3.7700734 ,   7.8073134 , -23.333174  , ...,  -6.3825936 ,
         18.216936  ,  25.894047  ]], dtype=float32)

In [ ]:
data = pd.DataFrame(tmp, index=paradigm.index)


### E

In [14]:
from numrisk.utils import Subject
bids_folder = '/Users/mrenke/data/ds-dnumrisk'
session = 1
n_stim = 2 # we are interested in activity from the second stimulus
base_dir = f'glm_stim{n_stim}.denoise'
runs = range(1, 7)
subject_id = 1
subject = f'{int(subject_id):02d}'
sub = Subject(subject_id, bids_folder=bids_folder)



In [10]:
masker = sub.get_brain_masker()

In [35]:
import nibabel as nib
import os.path as op
target_dir = '/Users/mrenke/data/ds-dnumrisk/derivatives/encoding_model.2d.cv.mixture.same_rfs/sub-01/func'
run=1

cv_r2s = []
for run in range(1, 7):
    fn = op.join(target_dir, f'sub-{subject}_ses-{session}_task-magjudge_run-{run}_desc-gd.cvr2_space-T1w_pars.nii.gz')
    cv_r2_gd_raw = nib.load(fn)
    cv_r2_gd = masker.fit_transform(cv_r2_gd_raw)
    cv_r2s.append(cv_r2_gd[0])

In [36]:
import pandas as pd
cv_r2s_dfs = [pd.DataFrame(arr) for arr in cv_r2s]

cv_r2s_ = pd.concat(cv_r2s_dfs, keys=runs, names=['run']).groupby(level=1, axis=0).mean()


In [42]:
cv_r2s_ = pd.concat(cv_r2s_dfs, keys=runs, names=['run']).groupby(level=1, axis=0).mean()

In [45]:
masker.inverse_transform(cv_r2s_.T).to_filename(op.join(target_dir, f'sub-{subject}_task-magdudge_desc-cvr2_space-T1w_pars.nii.gz'))


In [58]:
pars_gd = sub.get_cv_prf_parameters_volume(session=1,holdout_run=1, two_dimensional=True,
                                  mixture_model=True,return_image=False,
                                    same_rfs=True,
                                    roi='NPC_R')

/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/image/image.py:756: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  return klass(data, affine, header=header)
/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/image/resampling.py:598: RuntimeWarning: NaNs or infinite values are present in the data passed to resample. This is a bad thing as they make resampling ill-defined and much slower.
  _resample_one_img(data[all_img + ind], A, b, target_shape,
/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/nilearn/image/image.py:756: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  return klass(data, affine, h

In [59]:
n_voxels = 100

r2_mask = pars_gd['r2'].sort_values(ascending=False).index[:n_voxels]


In [55]:
r2_mask

Int64Index([ 7784,  7805,  7806,  7785,  9019, 11815,  8590,  8575,  8222,
             3087,  8223,  3078,  3086,  3077,  7804,  8591,  9039,  7783,
             7081,  3291,  9828,  3101,  8576,  8199,  3088,  3100, 11833,
             3127,  8251,  6493,  2884,  3300,  8250,  9020,  3292,  3114,
            11388,  2872,  9827,  3350, 11337,  3079, 11738,  3301, 11363,
             3335,  7807,  7756,  8224, 12235,  6492, 11816,  3113,  7822,
             9040, 11737,  8574,  9846,  9844,  3336,  7823,  9845,  3068,
             8589,  3085,  2862,  6494,   955,  7446,  3069,  6048,  7757,
             3126,  3137, 12217,  1228,  8252,  7447, 11834,  9865,  3099,
             9337,  9843,  3349,  2861,   971,  8249,  3076,  6474,  9826,
             9311, 10056,  8200, 14961,  1229,  9001,  9310, 10057,  6047,
             9026],
           dtype='int64')

In [62]:
sub = Subject(subject_id, bids_folder)
behavior = sub.get_behavior_magjudge()
paradigm = np.log(behavior[['n1', 'n2']].copy()).dropna().droplevel('subject')

data = sub.get_single_trial_volume(session, n_stim=n_stim, roi=None,smoothed=False, denoise=True).astype(np.float32)
data.index = paradigm.index

In [63]:
test_run = 1
test_data, test_paradigm = data.loc[test_run].copy(), paradigm.loc[test_run].copy()
train_data, train_paradigm = data.drop(test_run, level='run').copy(), paradigm.drop(test_run, level='run').copy()


In [64]:
train_data

0          1         2         3          4         5      \
run trial_nr                                                                 
2   31        4.467721  -1.448377 -3.355821  3.953604  -7.596740  4.145755   
    32       -1.635356 -10.795338 -5.356212  7.312120  -1.371064  0.057888   
    33        0.785027   0.249403  4.847545  0.036259   4.467438 -3.707404   
    34       -0.714537   5.172987  2.770771  0.484973 -12.024037  6.259865   
    35       -1.962139   4.060481  3.044915 -5.075397  -7.883848 -3.373917   
...                ...        ...       ...       ...        ...       ...   
6   176       0.844216   1.886107  4.471626  8.928157  -6.495096  4.848532   
    177      -1.025549  -1.657252 -1.154629  1.715727  -3.363083  0.448298   
    178      -0.016821   2.196058  1.629169  0.269890  -4.450286 -1.978234   
    179       1.803620   2.956554 -0.256786 -0.285315   5.534694  7.251609   
    180      -1.229209  -1.178985  2.816704  4.001599   4.138150  0.271831   

                 6         7         8         9      ...     78788     78789  \
run trial_nr                                          ...                       
2   31        2.735186  0.548995 -8.625731 -2.094081  ...  4.760605 -1.314916   
    32       -2.909039 -2.389971 -0.270734 -0.217150  ...  2.773489 -1.216424   
    33        0.847298  2.567118  1.345955  0.123207  ... -6.761909  2.314336   
    34        2.215527  3.979806  2.018037 -2.247368  ... -4.336075  2.038765   
    35        1.343904  0.868449  4.181017 -3.431128  ... -4.429418 -0.187395   
...                ...       ...       ...       ...  ...       ...       ...   
6   176      -0.589672 -2.451388  1.072884  0.379037  ...  3.861291 -0.152373   
    177       1.172727  0.471781 -2.575269 -2.070989  ... -5.079347 -0.695222   
    178      -0.264937 -1.731840  1.833429 -2.309157  ... -1.772224 -1.055165   
    179       2.032198  3.183022 -7.417245  2.130675  ...  7.226232 -0.133155   
    180      -0.892062  1.838048  6.644657 -1.648669  ... -0.270348 -1.057706   

                 78790     78791     78792     78793     78794     78795  \
run trial_nr                                                               
2   31        2.145372  2.794543 -2.539463 -1.630210 -0.572249  2.048245   
    32       -0.745951 -3.675452 -8.883361 -0.596084  3.401952 -0.597486   
    33       -5.373157 -7.632072 -9.059211  0.402763  0.528217 -3.727443   
    34       -1.375362 -1.444064  4.746146 -0.687062  0.693766  0.880469   
    35       -1.303337 -1.416826  1.669030 -1.275944  1.034996 -1.209104   
...                ...       ...       ...       ...       ...       ...   
6   176       0.985013  2.461055  7.649776 -0.992782  0.183668  4.536236   
    177      -0.501963  0.579878 -1.410680  0.428011  2.761935 -0.355043   
    178       2.349221 -0.027559  0.961282 -0.728046  2.959388  2.516572   
    179      -2.664355 -1.224951 -0.498028 -0.466137 -2.434206  3.352535   
    180      -2.721148  2.732806  2.744406  2.453762 -3.253279  1.609859   

                 78796     78797  
run trial_nr                      
2   31       -0.597345  0.648026  
    32       -0.007028 -0.391585  
    33        0.966133  2.840067  
    34        0.699342  3.802534  
    35        1.697803  2.716034  
...                ...       ...  
6   176       7.510978  0.353446  
    177      -1.677738  3.269469  
    178      -2.586582  0.110352  
    179      -5.465135 -6.375852  
    180      -1.473159 -2.882010  

[150 rows x 78798 columns]

In [ ]:
pars = pars_gd.loc[r2_mask]
train_data = train_data[r2_mask].astype(np.float32)

In [67]:
from braincoder.models import GaussianPointPRF2D, GaussianMixturePRF2D, LogGaussianPRF
from braincoder.optimize import ParameterFitter, ResidualFitter

x = np.log(np.arange(5, 26, 1.))
y = x.copy()
paradigm_x, paradigm_y = np.meshgrid(x, y)

# Stack into a 10,000 x 2 array
stimulus_range = np.column_stack((paradigm_x.ravel(), paradigm_y.ravel()))

model = GaussianMixturePRF2D(same_rfs=True)

model.init_pseudoWWT(stimulus_range, pars)   

residfit = ResidualFitter(model, train_data, train_paradigm.astype(np.float32),
                    parameters=pars)

/Users/mrenke/mambaforge/envs/numrefields/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-06 14:46:14.527072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-06 14:46:14.527310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-11-06 14:46:14.721414: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-11-06 14:46:14.724637: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [68]:
omega, dof = residfit.fit(init_sigma2=10.0,
        init_dof=10.0,
        method='t',
        learning_rate=0.05,
        max_n_iterations=20000)

print('DOF', dof)

pdf = model.get_stimulus_pdf(test_data[r2_mask].astype(np.float32), omega=omega, dof=dof, stimulus_range=stimulus_range, normalize=False)


2024-11-06 14:46:33.465687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


init_tau: 0.635284960269928, 24.068159103393555
USING A PSEUDO-WWT!
WWT max: 45849.40234375


  0%|          | 0/20000 [00:00<?, ?it/s]2024-11-06 14:46:34.140993: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-06 14:46:34.292042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-06 14:46:34.564836: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-06 14:46:34.959803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-06 14:46:35.978929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-11-06 14:46:38.210022: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
fit stat: 26943.3848 (best: 2

DOF 58.014156


2024-11-06 14:48:31.622753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
behavior = sub.get_behavior_magjudge()
paradigm = np.log(behavior[['n1', 'n2']].copy()).dropna().droplevel('subject')

# get brain data
mask = op.join(bids_folder, 'derivatives', 'fmriprep', f'sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-1_space-T1w_desc-brain_mask.nii.gz')
if not op.exists(mask): # for some subs brain mask creation for some runs did not work apparently, so lets just take another run (they should be pretty similar?)
    mask = op.join(bids_folder, 'derivatives',
            f'fmriprep/sub-{subject}/ses-{session}/func/sub-{subject}_ses-{session}_task-magjudge_run-2_space-T1w_desc-brain_mask.nii.gz')
masker = NiftiMasker(mask_img=mask)
data = op.join(bids_folder, 'derivatives', base_dir, # base-dir defines we get actuvity from stim2
                                        f'sub-{subject}', f'ses-{session}', 'func', f'sub-{subject}_ses-{session}_task-magjudge_space-T1w_desc-stims{n_stim}_pe.nii.gz')
data = pd.DataFrame(masker.fit_transform(data), index=paradigm.index)
data = pd.DataFrame(data, index=paradigm.index)

test_run=2
test_data, test_paradigm = data.loc[test_run].copy(), paradigm.loc[test_run].copy()
train_data, train_paradigm = data.drop(test_run, level='run').copy(), paradigm.drop(test_run, level='run').copy()
